Facebook Prophet approach

In [ ]:
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

# Prepare the data
df = df.rename(columns={'Order Date': 'ds', 'Number of products sold': 'y'})

# Initialize the Model
model = Prophet()

# Fit the Model
model.fit(df)

# Create an empty dataframe for future predictions
future = model.make_future_dataframe(periods=365)

# Make predictions
forecast = model.predict(future)

# Calculate RMSE
y_true = df['y'].values
y_pred = forecast['yhat'][:len(df)].values
mse = mean_squared_error(y_true, y_pred)
rmse = math.sqrt(mse)
print('RMSE: %.2f' % rmse)


DeepAR approach

In [ ]:
!pip install sagemaker

In [ ]:
# import necessary libraries
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

# Set up the Sagemaker environment
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Define the S3 bucket and prefix that you want to use for training and model data
bucket = sagemaker_session.default_bucket()  # replace with your own bucket name if not using the default bucket
prefix = 'sagemaker/DEMO-deepar' # replace with the prefix under which you want to store the data if needed

# We use the DeepAR container
image_name = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')

# Define the estimator
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    base_job_name='DEMO-deepar',
    output_path="s3://{}/{}/output".format(bucket, prefix),
)

# Set the hyperparameters
estimator.set_hyperparameters(
    time_freq='D', # daily frequency
    context_length=30, # number of data points that the model gets to see before making a prediction
    prediction_length=30, # number of data points that the model is trained to predict
    epochs=50,
    early_stopping_patience=10,
    mini_batch_size=64,
    learning_rate=0.001,
)

# Specify the data channels
data_channels = {
    "train": "s3://{}/{}/train/".format(bucket, prefix),
    "test": "s3://{}/{}/test/".format(bucket, prefix)
}

# Fit the model
estimator.fit(inputs=data_channels)


In [ ]:
import pandas as pd
import random

class SKU:
    def __init__(self, sku_name, quantity, delivery_time):
        self.sku_name = sku_name
        self.quantity = quantity
        self.delivery_time = delivery_time

    def decrease_quantity(self):
        decrease_amount = random.randint(1, 10)  # Randomly decrease quantity by 1 to 10 units
        self.quantity -= decrease_amount
        if self.quantity < 0:
            self.quantity = 0

    def check_quantity(self):
        return self.quantity

def simulate_quantities(skus, num_iterations):
    for i in range(num_iterations):
        print(f"\nIteration {i + 1}:")
        for sku in skus:
            sku.decrease_quantity()
            print(f"{sku.sku_name}: Quantity - {sku.check_quantity()}")

def order_products(skus, sku_name, quantity):
    for sku in skus:
        if sku.sku_name == sku_name:
            print(f"Ordered {quantity} units of {sku_name}.")
            sku.quantity += quantity
            break
    else:
        print("SKU not found.")

def main():
    # Sample DataFrame with SKU names, quantities, and delivery times
    data = {
        'SKU Name': ['SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5'],
        'Quantity': [100, 150, 80, 200, 120],
        'Delivery Time (days)': [3, 5, 2, 4, 6]
    }
    df = pd.DataFrame(data)

    # Convert DataFrame to list of SKU objects
    skus = [SKU(row['SKU Name'], row['Quantity'], row['Delivery Time (days)']) for index, row in df.iterrows()]

    # Simulate quantities for 10 iterations
    simulate_quantities(skus, 10)

    # Order products based on user input
    while True:
        choice = input("\nDo you want to order products? (yes/no): ").lower()
        if choice == 'yes':
            sku_name = input("Enter SKU Name: ")
            quantity = int(input("Enter quantity to order: "))
            order_products(skus, sku_name, quantity)
        elif choice == 'no':
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please enter 'yes' or 'no'.")

    # Find SKU with minimum quantity and its delivery time
    min_sku = min(skus, key=lambda x: x.quantity)
    print(f"\nSKU with minimum quantity: {min_sku.sku_name}")
    print(f"Quantity: {min_sku.quantity}")
    print(f"Delivery Time (days): {min_sku.delivery_time}")

if __name__ == "__main__":
    main()


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Sample DataFrame with SKU names, quantities, and delivery times
data = {
    'SKU Name': ['SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5'],
    'Quantity': [100, 150, 80, 200, 120],
    'Delivery Time (days)': [3, 5, 2, 4, 6]
}
df = pd.DataFrame(data)

# Convert DataFrame to list of SKU objects
skus = [SKU(row['SKU Name'], row['Quantity'], row['Delivery Time (days)']) for index, row in df.iterrows()]

# Function to handle order button click
def order_button_click(button):
    sku_name = dropdown_sku.value
    quantity = int(quantity_text.value)
    order_products(skus, sku_name, quantity)

# GUI components
dropdown_sku = widgets.Dropdown(
    options=[sku.sku_name for sku in skus],
    description='SKU Name:',
    disabled=False,
)

quantity_text = widgets.IntText(
    value=0,
    description='Quantity:',
    disabled=False
)

button_order = widgets.Button(description="Order")
button_order.on_click(order_button_click)

output_text = widgets.Output()

# Display GUI components
display(dropdown_sku, quantity_text, button_order, output_text)


Dash

In [ ]:
!pip install dash

In [ ]:
import dash
from dash import dcc
from dash import html

app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Inventory Dashboard'),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': list(inventory.inventory.keys()), 'y': [item['stock_count'] for item in inventory.inventory.values()], 'type': 'bar', 'name': 'Inventory'},
            ],
            'layout': {
                'title': 'Inventory Levels'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)
